# 🌾 Consultas y Extracción de Datos de API Pública 


**Endpoint Principal:**
- URL Testing: `https://test.bc.org.ar/SiogranosAPI/api/ConsultaPublica/consultarOperaciones`
- Método: GET


## Importar librerías

In [2]:
import requests
import json

## Consultas a la API y Experimentación

### Prueba simple del endpoint

Consulta básica con un rango de fechas pequeño


In [3]:
# URL base del endpoint
url = "https://test.bc.org.ar/SiogranosAPI/api/ConsultaPublica/consultarOperaciones"

# Parámetros mínimos: consultar operaciones de un solo día
params = {
    'FechaOperacionDesde': '2025-06-01',
    'FechaOperacionHasta': '2025-06-01'
}

print(f"URL: {url}")
print(f"Parámetros: {params}")
print()

# Hacer la petición
response = requests.get(url, params=params)

# Ver el resultado
print(f"Status Code: {response.status_code}")
print(f"Tamaño de respuesta: {len(response.content)} bytes")


URL: https://test.bc.org.ar/SiogranosAPI/api/ConsultaPublica/consultarOperaciones
Parámetros: {'FechaOperacionDesde': '2025-06-01', 'FechaOperacionHasta': '2025-06-01'}

Status Code: 403
Tamaño de respuesta: 7355 bytes


Un 403 significa "Forbidden" (prohibido), pero tiene contenido.

In [9]:
# Ver el contenido de la respuesta
print("Contenido de la respuesta:")
print(response.text[:1000])  # Primeros 1000 caracteres
print("\n---")

# Ver los headers de la respuesta (pueden dar pistas)
print("\nHeaders importantes de la respuesta:")
for header in ['Content-Type', 'Server', 'WWW-Authenticate']:
    if header in response.headers:
        print(f"  {header}: {response.headers[header]}")


Contenido de la respuesta:
<!DOCTYPE html><html lang="en-US"><head><title>Just a moment...</title><meta http-equiv="Content-Type" content="text/html; charset=UTF-8"><meta http-equiv="X-UA-Compatible" content="IE=Edge"><meta name="robots" content="noindex,nofollow"><meta name="viewport" content="width=device-width,initial-scale=1"><style>*{box-sizing:border-box;margin:0;padding:0}html{line-height:1.15;-webkit-text-size-adjust:100%;color:#313131;font-family:system-ui,-apple-system,BlinkMacSystemFont,"Segoe UI",Roboto,"Helvetica Neue",Arial,"Noto Sans",sans-serif,"Apple Color Emoji","Segoe UI Emoji","Segoe UI Symbol","Noto Color Emoji"}body{display:flex;flex-direction:column;height:100vh;min-height:100vh}.main-content{margin:8rem auto;padding-left:1.5rem;max-width:60rem}@media (width <= 720px){.main-content{margin-top:4rem}}.h2{line-height:2.25rem;font-size:1.5rem;font-weight:500}@media (width <= 720px){.h2{line-height:1.5rem;font-size:1.25rem}}#challenge-error-text{background-image:url("

### Intento con headers de navegador real

El problema es que Cloudflare detecta que somos un bot. Vamos a agregar headers más completos para simular un navegador:


In [10]:
# Headers más completos simulando un navegador Chrome
headers = {
    'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36',
    'Accept': 'application/json, text/plain, */*',
    'Accept-Language': 'es-ES,es;q=0.9,en;q=0.8',
    'Accept-Encoding': 'gzip, deflate, br',
    'Referer': 'https://test.bc.org.ar/',
    'Origin': 'https://test.bc.org.ar',
    'Connection': 'keep-alive',
}

# Hacer la petición con los nuevos headers
response2 = requests.get(url, params=params, headers=headers)

print(f"Status Code: {response2.status_code}")
print(f"Tamaño de respuesta: {len(response2.content)} bytes")

if response2.status_code == 200:
    print("Funcionó!")
else:
    print(f"Todavía {response2.status_code}")
    print(f"Contenido: {response2.text[:200]}")


Status Code: 403
Tamaño de respuesta: 4768 bytes
Todavía 403
Contenido: � d��Z���s�wnmL���g�&JE$9^)}�\����d�F�� ����"P�R����H�\�����բ�VV�>��N�jt$�����ʦ��j��Kܼ���O�pS�kʧ*�)��t�O��� @E�	�i����Na�c<}� ���6�(��l����u����bM;i~�[�6��B��� ����Z���?�A��Я�oh��


Probar con una semana de datos

Sabemos que funciona desde Postman. Intentemos con el rango de una semana como en la documentación:


In [15]:
# Probar con una semana de datos (como en la documentación)
params_semana = {
    'FechaOperacionDesde': '2025-06-01',
    'FechaOperacionHasta': '2025-06-06',
    'IDTipoCondicionPago': 3  # Contra entrega
}

print("Probando con una semana de datos...")
response3 = requests.get(url, params=params_semana, headers=headers)

print(f"Status Code: {response3.status_code}")

if response3.status_code == 200:
    print("Funcionó con una semana!")
    print(f"Tamaño: {len(response3.content)} bytes")
else:
    print(f"Sigue bloqueado: {response3.status_code}")


Probando con una semana de datos...
Status Code: 403
Sigue bloqueado: 403


Cloudflare nos bloquea desde Python pero no desde Postman
Necesitamos usar 'cloudscraper' para bypassear Cloudflare

### Usar cloudscraper para bypassear Cloudflare

Cloudscraper es como requests pero bypasea automáticamente la protección de Cloudflare:


In [12]:
import cloudscraper

# Crear un scraper (funciona igual que requests)
scraper = cloudscraper.create_scraper()

# Probar con una semana
print("Intentando con cloudscraper...")
response_cs = scraper.get(url, params=params_semana)

print(f"Status Code: {response_cs.status_code}")
print(f"Tamaño: {len(response_cs.content)} bytes")

if response_cs.status_code == 200:
    print("FUNCIONÓ con cloudscraper!")
else:
    print(f"Error: {response_cs.status_code}")


Intentando con cloudscraper...
Status Code: 200
Tamaño: 480302 bytes
FUNCIONÓ con cloudscraper!


### Ver el contenido del JSON

Ahora que funciona, veamos qué datos nos devuelve:


In [13]:
# Parsear el JSON
datos = response_cs.json()

# Ver la estructura principal
print("Estructura del JSON:")
print(f"  - Claves principales: {list(datos.keys())}")
print()

# Ver si tiene el campo 'success' y 'message'
if 'success' in datos:
    print(f"Success: {datos['success']}")
    print(f"Message: {datos['message']}")
    print()

# Ver cuántas operaciones devolvió
if 'result' in datos and 'operaciones' in datos['result']:
    operaciones = datos['result']['operaciones']
    print(f"Total de operaciones: {len(operaciones)}")
else:
    print("No se encontró el campo 'operaciones'")
    print(f"Contenido completo: {datos}")


Estructura del JSON:
  - Claves principales: ['success', 'message', 'result']

Success: True
Message: exito

Total de operaciones: 720


## Guardar los datos en JSON

Guardemos los datos en un archivo para poder trabajar con ellos:


In [16]:
import os

# Crear directorio si no existe
os.makedirs('../data/raw', exist_ok=True)

# Guardar los datos completos
output_file = '../data/raw/operaciones_semana_2025-06-01_2025-06-06.json'

with open(output_file, 'w', encoding='utf-8') as f:
    json.dump(datos, f, indent=2, ensure_ascii=False)

print(f"Datos guardados en: {output_file}")
print(f"Total de operaciones guardadas: {len(operaciones)}")


Datos guardados en: ../data/raw/operaciones_semana_2025-06-01_2025-06-06.json
Total de operaciones guardadas: 720


### Descargar las 3 semanas anteriores:


In [18]:
from datetime import datetime, timedelta
import time

# Definir las semanas a descargar (anteriores a la que ya tenemos)
semanas = [
    ('2025-05-25', '2025-05-31'),  # Semana anterior
    ('2025-05-18', '2025-05-24'),  # 2 semanas atras
    ('2025-05-11', '2025-05-17'),  # 3 semanas atras
]

for fecha_desde, fecha_hasta in semanas:
    print(f"\nDescargando: {fecha_desde} a {fecha_hasta}...")
    
    # Parametros para esta semana
    params = {
        'FechaOperacionDesde': fecha_desde,
        'FechaOperacionHasta': fecha_hasta,
        'IDTipoCondicionPago': 3
    }
    
    # Hacer la peticion
    response = scraper.get(url, params=params)
    
    if response.status_code == 200:
        datos_semana = response.json()
        num_ops = len(datos_semana['result']['operaciones'])
        
        # Guardar archivo
        filename = f'../data/raw/operaciones_semana_{fecha_desde}_{fecha_hasta}.json'
        with open(filename, 'w', encoding='utf-8') as f:
            json.dump(datos_semana, f, indent=2, ensure_ascii=False)
        
        print(f"  OK - {num_ops} operaciones guardadas")
    else:
        print(f"  ERROR - Status {response.status_code}")
    
    # Esperar un poco entre peticiones para no saturar el servidor
    time.sleep(2)

print("\nDescarga completa!")



Descargando: 2025-05-25 a 2025-05-31...
  OK - 2243 operaciones guardadas

Descargando: 2025-05-18 a 2025-05-24...
  OK - 2261 operaciones guardadas

Descargando: 2025-05-11 a 2025-05-17...
  OK - 2164 operaciones guardadas

Descarga completa!


### Descargar 2 semanas más anteriores:


In [21]:
# 2 semanas mas anteriores
semanas_extra = [
    ('2025-05-04', '2025-05-10'),  # 4 semanas atras
    ('2025-04-27', '2025-05-03'),  # 5 semanas atras
]

for fecha_desde, fecha_hasta in semanas_extra:
    print(f"\nDescargando: {fecha_desde} a {fecha_hasta}...")
    
    params = {
        'FechaOperacionDesde': fecha_desde,
        'FechaOperacionHasta': fecha_hasta,
        'IDTipoCondicionPago': 3
    }
    
    response = scraper.get(url, params=params)
    
    if response.status_code == 200:
        datos_semana = response.json()
        num_ops = len(datos_semana['result']['operaciones'])
        
        filename = f'../data/raw/operaciones_semana_{fecha_desde}_{fecha_hasta}.json'
        with open(filename, 'w', encoding='utf-8') as f:
            json.dump(datos_semana, f, indent=2, ensure_ascii=False)
        
        print(f"  OK - {num_ops} operaciones guardadas")
    else:
        print(f"  ERROR - Status {response.status_code}")
    
    time.sleep(2)

print("\nDescarga completa!")



Descargando: 2025-05-04 a 2025-05-10...
  OK - 2144 operaciones guardadas

Descargando: 2025-04-27 a 2025-05-03...
  OK - 1478 operaciones guardadas

Descarga completa!


### Ver una operacion de ejemplo

Veamos los campos de una operacion para entender que datos tenemos:


In [22]:
# Ver la primera operacion
primera_op = operaciones[0]

print("Primera operacion:")
print(json.dumps(primera_op, indent=2, ensure_ascii=False))


Primera operacion:
{
  "fechaOperacion": "2025-06-03T12:59:42.993",
  "fechaConcertacion": "2025-05-09T00:00:00",
  "tipoOperacion": "Fijación",
  "tipoModalidad": "Compraventa",
  "tipoOiv": "Fijar Precio/Prec. Cam.",
  "grano": "MAIZ",
  "volumenTN": "250.00",
  "condicionCalidad": "Cámara",
  "condicionCalidadAuxiliar": "",
  "procedenciaProvincia": "SANTA FE",
  "procedenciaLocalidad": "LUIS PALACIOS",
  "simboloPrecioPorTN": "$",
  "precioTN": "205000.00",
  "lugarEntrega": "Rosario S/En destino",
  "fechaEntregaDesde": "2025-05-09T00:00:00",
  "fechaEntregaHasta": "2025-05-20T00:00:00",
  "condicionPago": "A plazo",
  "esDestinoFinal": "SI",
  "cosecha": "COSECHA 24/25",
  "numeroOperacionInstancia": "3",
  "nroOperacion": "2118370255",
  "esUltimaInstancia": "Si"
}
